In [61]:
#Import

#Modelling
from mesa import Agent, Model

#Analysis
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid
from mesa.time import RandomActivation

from mesa.experimental import JupyterViz

In [62]:
from typing import Any

#Model

class MoneyModel(Model):
    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            model_reporters={
            "Gini coefficient": compute_gini #function defined below
        },
        agent_reporters={
            "Agent wealth": "wealth",
            "Agent position": "pos",
        }, #agent attributes
    )
        
    # Create agents
        for unique_id in range(self.num_agents):
            agent = MoneyAgent(unique_id, self)
            self.schedule.add(agent)

            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(agent, (x, y))

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

#Agent

class MoneyAgent(Agent):
    def __init__(self, unique_id, model):
        # run the __init__ method of the parent class
        super().__init__(unique_id, model)
        # additional initialization for the derived class
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,  # current position
            moore=True,  # including step on the diagonal
            include_center=False,  # to remain at current position is not allowed
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

#Gini coefficient

def compute_gini(model):
    N = model.num_agents

    # collect wealth of agent into a list and sort it in increasing order
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)

    # calculate the Gini coefficient
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / sum(x)
    G = (N + 1 - 2 * B) / N

    return G

In [63]:
#We run the model with 50 agents distributed in a 10x10 grid of cells for 100 time steps.

model = MoneyModel(50, 10, 10)

for i in range(100):
    model.step()

In [64]:
#Collecting stored data

model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

model_df.head(5)

,Gini coefficient
0,0.0000
1,0.2744
2,0.3808
3,0.4824
4,0.5304


In [65]:
agent_df.tail(4)

Agent wealth Agent position
Step AgentID                             
99   46                  3         (2, 8)
     47                  0         (1, 3)
     48                  0         (4, 3)
     49                  1         (7, 6)

In [66]:
import matplotlib.pyplot as plt

model_df.plot(color="r", lw=2)

plt.ylabel("Gini coefficient")
plt.xlabel("Time step")
plt.grid("on")
plt.setp(plt.gca(), xlim=(-1, 100), ylim=(0, 1))

plt.show()

C:\Users\Savva\AppData\Local\Temp\ipykernel_8472\1384281656.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [67]:
# extract a cross section of the dataframe for the final time step
final_wealth_df = agent_df.xs(99, level="Step")

# plot a histogram
final_wealth_df.hist(bins=range(11))

plt.title("Wealth distribution")
plt.ylabel("Number agents")
plt.xlabel("Wealth")
plt.setp(
    plt.gca(), xlim=(0, 10), xticks=range(11), ylim=(0, 30), yticks=range(0, 31, 2)
)

plt.show()

C:\Users\Savva\AppData\Local\Temp\ipykernel_8472\515344154.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [68]:
step_idx, agent_idx = agent_df.idxmax().iloc[0]

print("Agent:", agent_idx)
print("Step:", step_idx)

Agent: 12
Step: 88


In [69]:
one_agent_wealth_df = agent_df.xs(agent_idx, level="AgentID")

one_agent_wealth_df.plot(lw=2, color="b")

plt.ylabel(f"Wealth of Agent {agent_idx}")
plt.xlabel("Time step")

plt.grid(True)
plt.setp(
    plt.gca(),
    xlim=(0, 100),
    xticks=range(0, 101, 10),
    ylim=(0, 10),
    yticks=range(0, 11, 2),
)

plt.show()

c:\Users\Savva\anaconda3\envs\cb1020\Lib\site-packages\pandas\plotting\_matplotlib\core.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = self.plt.figure(figsize=self.figsize)
C:\Users\Savva\AppData\Local\Temp\ipykernel_8472\3714621781.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [70]:
def agent_portrayal(agent):
    portrayal = {"Shape": "circle", "Filled": "true", "r":0.5}

    if agent.wealth > 0:
        portrayal["Color"] = "green"
        portrayal["Layer"] = 0
    else:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.2

    return portrayal

model_params = {
    "N": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of Neurons",
        "min": 50,
        "max": 200,
        "step": 1,
    },
    "width":100,
    "height":100,
}

page = JupyterViz(
    MoneyModel,
    model_params,
    measures=["Gini coefficient"],
    name = "Brain Model",
    agent_portrayal=agent_portrayal,
)

page

Cannot show ipywidgets in text